# IBM's Applied Data Science Capstone Project on Coursera
<h1>Author - J Prince Kelvin</h1>

### Week 5 Final Project

### Opening a New shopping Mall in Bangalore,India.

<ul>
   <li>Load the DataSet of neighborhoods in Bangalore,India <br> Source-<a href="https://www.kaggle.com/hegdetapan/bangaloreareaspincodewithlatitudelongitude">https://www.kaggle.com/hegdetapan/bangaloreareaspincodewithlatitudelongitude</a>
     <ul>
         <li> Description: Contains a list of pin codes, area names, along with latitude and longitude coordinates.</li>
     </ul>
   </li>
    <li> Obtain the Venue Data in each postal code Using FourSquare API.</li>
    <li> Explore and Cluster the Neighborhoods Data.</li>
    <li> Determine the best CLuster( Location ) to Open a new Shopping Mall.</li>
</ul>

### Importing Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import json
import geocoder
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import folium
import matplotlib.cm as cm
import matplotlib.colors as color
import requests
print("Libraries Imported")

Libraries Imported


### Loading Dataset

In [2]:
df = pd.read_excel("Bangalore.xlsx")
print("Data has been successfully Imported")
df.head(20)

Data has been successfully Imported


,Area,Pincode,Latitude,Longitude
0,BangaloreG.P.O.,560001,12.972442,77.580643
1,BangaloreCityH.O.,560002,12.972442,77.580643
2,Malleswaram,560003,13.008100,77.564800
3,Vyalikaval,560003,13.004100,77.574900
4,Basavangudi,560004,12.942200,77.574800
5,FraserTown,560005,13.000700,77.616500
6,JCNagar,560006,12.987600,77.637900
7,Agaram,560007,12.843100,77.486300
8,UlsoorH.O.,560008,12.981500,77.619200
9,Gandhinagar,560009,12.979100,77.577700


In [3]:
df.shape

(90, 4)

#### Getting Lattitude and Longitube Coordinates Of Each Neighborhood
Even though our Dataset contains Lattitude and Longitube Coordinates , we will use geocoder to generate and get accurate Coordinate values of our Neighborhoods(Area).

In [4]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Bangalore, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords


In [5]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in df["Area"].tolist() ]

Status code Unknown from https://geocode.arcgis.com/arcgis/rest/services/World/GeocodeServer/find: ERROR - HTTPSConnectionPool(host='geocode.arcgis.com', port=443): Read timed out. (read timeout=5.0)


In [6]:
# Create a temporary dataframe to store our coordinate values
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [7]:
# Update the new Coordinates in our Original Dataframe
df['Latitude']=df_coords['Latitude']
df['Longitude']=df_coords['Longitude']

In [8]:
df.shape

(90, 4)

In [9]:
df.head()

,Area,Pincode,Latitude,Longitude
0,BangaloreG.P.O.,560001,12.887035,77.74105
1,BangaloreCityH.O.,560002,12.966180,77.58690
2,Malleswaram,560003,13.006320,77.56840
3,Vyalikaval,560003,13.037920,77.61863
4,Basavangudi,560004,12.941120,77.57236


#### Segmenting And clustering the neighborhoods of Bangalore using FourSquare API

In [10]:
#Getting geographical coordinates of Bangalore
address = 'Bangalore'

geolocator = Nominatim(user_agent="Bangalore_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bangalore are ',latitude, longitude)

The geograpical coordinate of Bangalore are  12.9791198 77.5912997


In [11]:
# Let us now visualize map of Bangalore

map_bangalore = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bangalore)  
    
map_bangalore

In [12]:
# save the map as HTML file
map_bangalore.save('Bangalore_map.html')

In [1]:
#Foursquare API credentials
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


### Let's get the top 100 venues that are in Parkwoods within a radius of 3000 meters

In [14]:
# Defining a Function
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'VenueLatitude', 
                  'VenueLongitude', 
                  'VenueCategory']
    
    return(nearby_venues)

In [15]:
# Storing the venues in a new data frame
bangalore_venues = getNearbyVenues(names=df['Area'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

In [16]:
print(bangalore_venues.shape)
bangalore_venues.head()

(5806, 7)


,Neighborhood,Latitude,Longitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
0,BangaloreG.P.O.,12.887035,77.74105,Rural Blues,12.897550,77.717336,Bar
1,BangaloreG.P.O.,12.887035,77.74105,Jockey Factory Outlet,12.885835,77.743717,Clothing Store
2,BangaloreG.P.O.,12.887035,77.74105,Red Riders Sports,12.879376,77.737610,Go Kart Track
3,BangaloreG.P.O.,12.887035,77.74105,Priyadharshini Hotel,12.883636,77.751480,Fast Food Restaurant
4,BangaloreG.P.O.,12.887035,77.74105,HDFC Bank,12.882041,77.755478,ATM


In [17]:
#Let's find out how many venues were returned for each neighborhood
bangalore_venues.groupby('Neighborhood').count()

,Latitude,Longitude,Venue,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Adugodi,100,100,100,100,100,100
Agaram,4,4,4,4,4,4
AirForceStn.Yelahanka,7,7,7,7,7,7
Banashankari,100,100,100,100,100,100
Banashankari2ndStage,100,100,100,100,100,100
...,...,...,...,...,...,...
Vyalikaval,38,38,38,38,38,38
Whitefield,89,89,89,89,89,89
WilsonGarden,100,100,100,100,100,100


In [18]:
# Let's find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(bangalore_venues['VenueCategory'].unique())))

There are 195 uniques categories.


In [19]:
# print out the list of categories
bangalore_venues['VenueCategory'].unique()[:50]

array(['Bar', 'Clothing Store', 'Go Kart Track', 'Fast Food Restaurant',
       'ATM', 'Indoor Play Area', 'Café', 'Bus Station', 'Garden Center',
       'Restaurant', 'Hotel', 'Shopping Mall', 'Japanese Restaurant',
       'French Restaurant', 'Italian Restaurant', 'Deli / Bodega', 'Park',
       'Sushi Restaurant', 'Arcade', 'Lounge', 'Cupcake Shop',
       'Furniture / Home Store', 'Ice Cream Shop', 'Brewery',
       'Burger Joint', 'Indian Restaurant', 'Fried Chicken Joint',
       'Vegetarian / Vegan Restaurant', 'Seafood Restaurant',
       'Gym / Fitness Center', 'Breakfast Spot', 'American Restaurant',
       'Cricket Ground', 'Pub', 'Movie Theater', 'Plaza', 'Dessert Shop',
       'Botanical Garden', 'Bakery', 'Bookstore', 'Snack Place',
       'Toy / Game Store', 'Cocktail Bar', 'Pizza Place', 'Tea Room',
       'Sandwich Place', 'Afghan Restaurant', 'Art Gallery',
       'Andhra Restaurant', 'BBQ Joint'], dtype=object)

In [20]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in bangalore_venues['VenueCategory'].unique()

True

### Analyze Each Neighborhood

In [21]:
# one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

print(bangalore_onehot.shape)
bangalore_onehot.head()

(5806, 195)


,ATM,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,...,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Now, Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
bangalore_grouped = bangalore_onehot.groupby('Neighborhood').mean().reset_index()
print(bangalore_grouped.shape)
bangalore_grouped.head()

(89, 195)


,Neighborhood,ATM,Afghan Restaurant,Airport,Airport Terminal,American Restaurant,Andhra Restaurant,Arcade,Art Gallery,Art Museum,...,Trail,Train Station,Travel & Transport,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Adugodi,0.0,0.0,0.0,0.0,0.0,0.01,0.01,0.0,0.0,...,0.00,0.0,0.0,0.02,0.0,0.0,0.0,0.00,0.01,0.0
1,Agaram,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.25,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0
2,AirForceStn.Yelahanka,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.00,0.0
3,Banashankari,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.01,0.00,0.0
4,Banashankari2ndStage,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,...,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.01,0.00,0.0


In [23]:
len(bangalore_grouped[bangalore_grouped["Shopping Mall"] > 0])

51

#### Let's Create a new Dataframe for storing Shopping Mall data only

In [24]:
bangalore_mall = bangalore_grouped[['Neighborhood','Shopping Mall']]

In [25]:
print(bangalore_mall.shape)
bangalore_mall.head()

(89, 2)


,Neighborhood,Shopping Mall
0,Adugodi,0.01
1,Agaram,0.00
2,AirForceStn.Yelahanka,0.00
3,Banashankari,0.00
4,Banashankari2ndStage,0.00


## Cluster Neighborhoods
### Run k-means to cluster the neighborhood

In [26]:
# set number of clusters
kclusters = 5

bangalore_clustering = bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangalore_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 4, 0, 0, 0, 0, 2, 4, 2, 0])

##### Create a new Data Frame including the clusters

In [27]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
bangalore_merged = bangalore_mall.copy()

# add clustering labels
bangalore_merged["Cluster Labels"] = kmeans.labels_

bangalore_merged = bangalore_merged.join(df.set_index('Area'), on='Neighborhood')

bangalore_merged.head() # check the last columns!

,Neighborhood,Shopping Mall,Cluster Labels,Pincode,Latitude,Longitude
0,Adugodi,0.01,2,560030,12.94401,77.60797
1,Agaram,0.00,4,560007,12.84292,77.48582
2,AirForceStn.Yelahanka,0.00,0,560063,13.13214,77.61619
3,Banashankari,0.00,0,560050,12.92231,77.56988
4,Banashankari2ndStage,0.00,0,560070,12.91833,77.57364


In [28]:
# sort the results by Cluster Labels
print(bangalore_merged.shape)
bangalore_merged.sort_values(["Cluster Labels"], inplace=True)
bangalore_merged

(89, 6)


,Neighborhood,Shopping Mall,Cluster Labels,Pincode,Latitude,Longitude
44,JayanagarSouth,0.010000,0,560041,12.930140,77.58496
25,Dooravaninagar,0.022222,0,560016,12.997445,77.66830
28,Gandhinagar,0.000000,0,560009,13.026430,77.61751
29,GavipuramExt.,0.000000,0,560019,12.945280,77.56634
30,GokulaExtension,0.019608,0,560054,13.041120,77.54829
...,...,...,...,...,...,...
60,Nagarbhavi,0.000000,4,560072,12.956240,77.50936
1,Agaram,0.000000,4,560007,12.842920,77.48582
51,Kumbalgodu,0.000000,4,560074,13.072610,77.50321
66,PeenyaS.I.,0.111111,4,560058,13.031880,77.52654


## Finally, Let us now Visualize our Clusters

In [29]:
#  map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [color.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['Latitude'], bangalore_merged['Longitude'], bangalore_merged['Neighborhood'], bangalore_merged['Cluster Labels']):
    if (cluster != cluster):
        cluster=0
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [30]:
# save the map as an HTML file
map_clusters.save('map_clusters.html')

### Let's examine each of our clusters

### Cluster 1

In [31]:
c1=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 0]
print("Total number of Shopping malls is =",c1['Shopping Mall'].count())
c1

Total number of Shopping malls is = 33


,Neighborhood,Shopping Mall,Cluster Labels,Pincode,Latitude,Longitude
44,JayanagarSouth,0.010000,0,560041,12.930140,77.584960
25,Dooravaninagar,0.022222,0,560016,12.997445,77.668300
28,Gandhinagar,0.000000,0,560009,13.026430,77.617510
29,GavipuramExt.,0.000000,0,560019,12.945280,77.566340
30,GokulaExtension,0.019608,0,560054,13.041120,77.548290
33,HebbalAgri.Farm,0.000000,0,560024,13.049810,77.589030
37,HospitalTownWest,0.023810,0,560015,12.905700,77.542720
43,JayanagarEast,0.010000,0,560069,12.919330,77.592690
87,Yelhanka,0.000000,0,560064,13.084356,77.594929
23,Doddakallasandra,0.000000,0,560062,12.886610,77.558720


### Cluster 2

In [32]:
c2=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 1]
print("Total number of Shopping malls is =",c2['Shopping Mall'].count())
c2

Total number of Shopping malls is = 3


,Neighborhood,Shopping Mall,Cluster Labels,Pincode,Latitude,Longitude
34,Hesaraghatta,0.0,1,560088,13.13928,77.47976
35,HesaraghattaLake,0.0,1,560089,13.15029,77.47639
36,HospitalTownEast,0.0,1,560014,13.15665,77.47514


### Cluster 3

In [33]:
c3=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 2]
print("Total number of Shopping malls is =",c3['Shopping Mall'].count())
c3

Total number of Shopping malls is = 38


,Neighborhood,Shopping Mall,Cluster Labels,Pincode,Latitude,Longitude
69,RichmondTown,0.010000,2,560025,12.962330,77.601230
68,Rajajinagar,0.020000,2,560010,13.005440,77.556930
22,DharmaramCollege,0.010000,2,560029,12.936750,77.602640
8,BangaloreUniversity,0.030000,2,560056,12.996940,77.618430
59,Mathikere,0.010000,2,560054,13.032340,77.558640
58,Maruthisevanagar,0.034091,2,560033,13.000130,77.627790
6,BangaloreCityH.O.,0.020000,2,560002,12.966180,77.586900
65,NewTippasandra,0.010000,2,560075,12.973600,77.665650
74,St.ThomasTown,0.030000,2,560084,13.005020,77.623250
71,Shanthinagar,0.010000,2,560027,12.957560,77.596930


### Cluster 4

In [34]:
c4=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 3]
print("Total number of Shopping malls is =",c4['Shopping Mall'].count())
c4

Total number of Shopping malls is = 1


,Neighborhood,Shopping Mall,Cluster Labels,Pincode,Latitude,Longitude
52,MagadiRoad,0.0,3,560023,12.996875,77.443065


###  Cluster 5

In [35]:
c5=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 4]
print("Total number of Shopping malls is =",c5['Shopping Mall'].count())
c5

Total number of Shopping malls is = 14


,Neighborhood,Shopping Mall,Cluster Labels,Pincode,Latitude,Longitude
38,Indiranagar,0.125000,4,560038,13.030060,77.495260
20,Chikkabanavara,0.000000,4,560090,13.072135,77.513750
64,Nayandahalli,0.000000,4,560039,12.942050,77.521000
49,Kothanur,0.000000,4,560077,13.064560,77.647820
7,BangaloreG.P.O.,0.000000,4,560001,12.887035,77.741050
46,Kengeri,0.000000,4,560060,12.908680,77.487180
80,Vijayanagar,0.000000,4,560040,13.076000,77.652400
17,Carmelram,0.000000,4,560035,12.909638,77.704379
41,Jalahalli,0.166667,4,560013,13.054500,77.526580
60,Nagarbhavi,0.000000,4,560072,12.956240,77.509360


## Conclusion

Most of the <b>Shopping malls </b>are concentrated in the central area of <b>Bangalore</b> city,with the highest number of malls in <b>Cluster 1 and 3 </b> <b><i>(more than 30 malls)</i></b>  and moderate number of malls in <b> Cluster 5</b><b><i>(which is around 14 malls)</i></b>.
On the other hand,<b> Cluster 2 and 4 </b> has very low number of malls.
<br>
This represents a great opportunity and high potential areas to open new Shopping Malls as there is very little to no competiiton from existing malls.Meanwhile, shopping malls in <b>Cluster 1 and 3 </b> are suffering from intense competion due to oversupply and high concentration of shopping malls.
<br>
From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls.
<br>
Therefore, this project recommends <b>Property Developers & Investors </b> to capitalize on these findings to open new shopping malls in neighborhoods in <b> Cluster 2 and 4 </b> with little to no competition.
<br>  Property developers with unique selling propositions to stand out from the competition can also open new shopping malls in neighborhoods in <b> Cluster 5</b> with moderate competition.
<br>
Lastly, <b>Property Developers & Investors </b> are advised to avoid neighborhoods in <b>Cluster 1 and 3 </b>  which already have high concentration of shopping malls and suffering from intense competition.